<a href="https://colab.research.google.com/github/jspumn/jspumn.github.io/blob/main/Post_Conference_Syndrome_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Conference Syndrome Simulator (Run this cell)
# 1. Install necessary libraries
!pip install gradio plotly pandas -q

import gradio as gr
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# --- 1. The Simulation Engine ---
def run_simulation(total_attendees, conf_duration, interaction_rate, ventilation_score, mask_usage):
    # Fixed Biological Constants (based on respiratory viruses like Flu/Covid)
    transmission_prob = 0.04
    incubation_period = 3
    initial_infected = 2
    recovery_rate = 0.2

    # Setup
    days = conf_duration + 14  # Simulate conference + 2 weeks post-travel
    S = total_attendees - initial_infected
    E = 0
    I = initial_infected
    R = 0

    # History for plotting
    history = []

    # Queue to track when exposed people become infectious (Incubation logic)
    # Format: [day_exposed, count]
    exposure_queue = []

    for day in range(1, days + 1):
        is_conference = day <= conf_duration

        # 1. Calculate Mitigation (Ventilation + Masks)
        # Score 0 to 1 (1 is perfect protection)
        mitigation_factor = (ventilation_score * 0.4) + (mask_usage * 0.5)
        effective_transmission = transmission_prob * (1 - mitigation_factor)

        # 2. Interactions drop dramatically after conference ends
        current_interactions = interaction_rate if is_conference else 2

        # 3. Calculate New Exposures
        # Standard Force of Infection formula
        new_exposed = 0
        if I > 0:
            force_of_infection = (I * current_interactions * effective_transmission) / total_attendees
            new_exposed = min(S, int(S * force_of_infection))

        if new_exposed > 0:
            exposure_queue.append({'day': day, 'count': new_exposed})
            S -= new_exposed
            E += new_exposed

        # 4. Move Exposed -> Infectious (after incubation)
        newly_infectious = 0
        for batch in exposure_queue:
            if day == batch['day'] + incubation_period:
                newly_infectious += batch['count']

        if newly_infectious > 0:
            E -= newly_infectious
            I += newly_infectious

        # 5. Recovery
        new_recovered = int(I * recovery_rate)
        I -= new_recovered
        R += new_recovered

        # Safety clamps
        if S < 0: S = 0
        if E < 0: E = 0
        if I < 0: I = 0

        history.append({
            'Day': day,
            'Status': 'Conference' if is_conference else 'Home/Travel',
            'Susceptible': S,
            'Incubating (Exposed)': E,
            'Sick (Infectious)': I,
            'Recovered': R
        })

    df = pd.DataFrame(history)

    # Calculate Stats
    total_infected_final = (total_attendees - df.iloc[-1]['Susceptible'])
    attack_rate = (total_infected_final / total_attendees) * 100
    peak_sick = df['Sick (Infectious)'].max()
    peak_day = df.loc[df['Sick (Infectious)'].idxmax()]['Day']

    # --- 2. Visualization (Plotly) ---
    fig = go.Figure()

    # Add Exposed Area
    fig.add_trace(go.Scatter(
        x=df['Day'], y=df['Incubating (Exposed)'],
        mode='lines', name='Incubating',
        stackgroup='one', fillcolor='orange', line=dict(width=0)
    ))

    # Add Infectious Area
    fig.add_trace(go.Scatter(
        x=df['Day'], y=df['Sick (Infectious)'],
        mode='lines', name='Sick (Infectious)',
        stackgroup='one', fillcolor='red', line=dict(width=0)
    ))

    # Add Conference Duration Line
    fig.add_vline(x=conf_duration + 0.5, line_width=2, line_dash="dash", line_color="black")
    fig.add_annotation(x=conf_duration/2, y=total_attendees*0.9, text="Conference", showarrow=False)
    fig.add_annotation(x=conf_duration + 3, y=total_attendees*0.9, text="Home/Travel", showarrow=False)

    fig.update_layout(
        title="Conference Outbreak Timeline",
        xaxis_title="Days",
        yaxis_title="People",
        template="plotly_white",
        hovermode="x unified",
        height=400
    )

    # Generate Stats Text
    stats_markdown = f"""
    ### 📊 Predicted Outcome
    * **Total Infected:** {int(total_infected_final)} people
    * **Attack Rate:** {attack_rate:.1f}%
    * **Peak Illness Day:** Day {peak_day}
    """

    if peak_day > conf_duration:
        stats_markdown += f"\n⚠️ **WARNING:** The peak occurs **after** travel, meaning attendees are spreading this globally."

    return fig, stats_markdown

# --- 3. The Dashboard GUI (Gradio) ---

with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# ✈️ Conference Syndrome Analyzer")
    gr.Markdown("Simulate how pathogens spread at international conferences and create global outbreaks.")

    with gr.Tab("Simulator"):
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 🎛️ Parameters")
                attendees = gr.Slider(100, 5000, value=1000, step=100, label="Total Attendees")
                duration = gr.Slider(1, 7, value=3, step=1, label="Conference Duration (Days)")
                interactions = gr.Slider(2, 50, value=20, step=1, label="Interactions per Person/Day")

                gr.Markdown("### 🛡️ Mitigation")
                ventilation = gr.Slider(0, 1, value=0.2, label="Ventilation Quality (0=Poor, 1=Outdoor)")
                masks = gr.Slider(0, 1, value=0.0, label="Mask Usage (0% to 100%)")

                btn = gr.Button("Run Simulation", variant="primary")

            with gr.Column(scale=2):
                plot_output = gr.Plot(label="Infection Curve")
                stats_output = gr.Markdown("### Run simulation to see results")

    with gr.Tab("Research Findings"):
        gr.Markdown("""
        ### The Science of "Con Crud"

        **1. The "Mixing Vessel" Effect**
        Conferences bring together unique viral strains from different continents. Your immune system may be ready for your local Flu strain, but not the one brought by a colleague from overseas.

        **2. The Biogen Case Study (2020)**
        A leadership conference in Boston with ~175 people resulted in an estimated **300,000 downstream infections**. Attendees got infected at the event, felt fine, flew home, and unknowingly spread the virus globally before symptoms started.

        **3. Environmental Fatigue**
        * **Dry Air:** Hotel HVAC systems dry out nasal mucus (your first defense).
        * **Stress:** Travel lag and lack of sleep suppress IgA (immune response).
        * **Vocalization:** Shouting over noise in exhibition halls releases 10x more aerosols than quiet talking.
        """)

    # Link inputs to function
    btn.click(
        fn=run_simulation,
        inputs=[attendees, duration, interactions, ventilation, masks],
        outputs=[plot_output, stats_output]
    )

# Launch the app
app.launch(debug=True)
```

### How to use this with GitHub

You have two main options for sharing this.

#### Option 1: The "Open in Colab" Method (Easiest)
This allows anyone to click a badge on your GitHub README and immediately run your simulation in their own browser.

1.  **Save the file:**
    * Copy the code above.
    * Go to [colab.research.google.com](https://colab.research.google.com).
    * Create a new notebook and paste the code.
    * Name it `Conference_Syndrome.ipynb`.

2.  **Save to GitHub:**
    * In Colab, go to **File** > **Save a copy in GitHub**.
    * Authorize your account and select your repository.

3.  **Add the Badge:**
    * In your GitHub repository's `README.md`, add this code at the top:
    ```markdown
    [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/Conference_Syndrome.ipynb)
    ```
    *(Replace `YOUR_USERNAME`, `YOUR_REPO` with your actual details).*

#### Option 2: Hugging Face Spaces (Professional Way)
If you want it to run like a real website (without the user seeing code), use Hugging Face.

1.  Create a free account at [huggingface.co](https://huggingface.co).
2.  Click **New Space**.
3.  Select **Gradio** as the SDK.
4.  Create a file named `app.py` in that Space.
5.  Paste the python code I provided above (remove the `!pip install` line at the very top, as Hugging Face installs dependencies automatically via a `requirements.txt`).
6.  Create a `requirements.txt` file and add:
    ```text
    pandas
    plotly

SyntaxError: unterminated string literal (detected at line 205) (ipython-input-285938957.py, line 205)